In [2]:
import pandas as pd
import numpy as np
import utils as ut
import matplotlib.pyplot as plt

path = "data/ARIA/5.CourseOfTreatment/"

show = 1
ef  = ut.leer_csv(path+"_ExternalField.csv", show).drop(columns=["EllipticalMarginFlag",
                                                                                     "DRRTemplateFileName","TrackingSer",
                                                                                    "CalculationLog","OptimizeCollRtnFlag",
                                                                                    "WedgeWeightFactor"])

efc = ut.leer_csv(path+"_ExternalFieldCommon.csv", show).drop(columns=["SourceFieldEntryDistance",
                                                                                           "IDUPosLat","IDUPosLng"])

rad = ut.leer_csv(path+"_Radiation.csv", show).drop(columns=["RefImageSOPClassSer","RefImageUID",
                                                                                 "SetupNote","RadiationId","RadiationName",
                                                                                "CreationUserName","Comment","RefImageSer",
                                                                                "CreationDate"])

path = "data/ARIA_merged&selected/"

show = 0
rtp = ut.leer_csv(path + "RTPlan_PS_Pres.csv", show)
tr = ut.leer_csv(path + "TreatmentRecord.csv", show)

Archivo: data/ARIA/5.CourseOfTreatment/_ExternalField.csv
Se dejaron 30 columnas.
Se eliminaron 12 columnas:




D:\miniconda3\envs\tfm2\lib\site-packages\IPython\core\interactiveshell.py:3361: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Archivo: data/ARIA/5.CourseOfTreatment/_ExternalFieldCommon.csv
Se dejaron 26 columnas.
Se eliminaron 12 columnas:


Archivo: data/ARIA/5.CourseOfTreatment/_Radiation.csv
Se dejaron 15 columnas.
Se eliminaron 5 columnas:




In [3]:
# ut.info_tabla(rad)

In [4]:
join = rad.merge(ef, on="RadiationSer", how="inner", suffixes = ("_rad", "_ef"))\
.merge(efc, on="RadiationSer", how="inner", suffixes = ("_radef", "_efc"))
join.shape

(77087, 51)

In [5]:
join.to_csv("data/ARIA_merged&selected/Radiation.csv", index=False)

# Radiation, valores agrupados por RTPlanSer (PlanSetupSer)

In [6]:
# ut.resumen_tabla(join)
# ut.info_tabla(join)
join.PlanSetupSer.value_counts()

# ut.info_tabla(join[join.PlanSetupSer==132187])
# ut.info_tabla(join)

132673    27
121371    23
95297     21
25391     20
27806     20
          ..
6237       1
4726       1
5312       1
117947     1
143817     1
Name: PlanSetupSer, Length: 14148, dtype: int64

In [7]:
join2 = join.drop(columns=["RadiationNumber","RadiationOrder","MotionCompSource","MotionCompTechnique","RefDose","WedgeDose",
                          "CollMarginBottom","CollMarginTop","CollMarginRight","CollMarginLeft","StopAngle","FldNormMethod",
                          "IDURtn","ResourceSer","CollY1","CollY2","CollX1","CollX2","CouchLat","CouchVrt","CouchLngDelta",
                           "CouchLatDelta","CouchVrtDelta","DirectionPointDistance","IsoCenterPositionX",
                           "IsoCenterPositionY","IsoCenterPositionZ","MUCoeff","IDUPosVrt","TreatmentTime","CouchLng",
                          "SetupTechnique","TechniqueLabel","Meterset","FldNormFactor","GantryRtnExt"])

join3 = join2[join2.PlanSetupSer.isin(tr.merge(rtp, on="RTPlanSer", how="left").PlanSetupSer)]
# ut.info_tabla(join2)
ut.resumen_tabla(join3, 'nunicos')

50984 registros
15 variables


,nans,nunicos,nunicos_r,dtype
SetupFieldFlag,0.00,2,0.00,int64
GantryRtnDirection,0.00,3,0.01,object
ToleranceSer,0.01,3,0.01,float64
CollMode,0.00,4,0.01,object
TechniqueSer,0.00,11,0.02,int64
DoseRate,0.10,17,0.03,float64
EnergyModeSer,0.10,19,0.04,float64
PatientSupportAngle,0.00,116,0.23,float64
GantryRtn,0.00,1053,2.07,float64
CollRtn,0.00,1592,3.12,float64


In [8]:
rad_grp = join2.groupby("PlanSetupSer").agg({'EnergyModeSer': "first", "TechniqueSer": "first", 'RadiationSer': 'count'})
rad_grp = rad_grp.rename(columns={"RadiationSer": "NumFields"})
rad_grp

,EnergyModeSer,TechniqueSer,NumFields
PlanSetupSer,,,
71,71.0,31,2
113,71.0,31,1
116,71.0,31,5
117,71.0,31,2
118,71.0,32,4
...,...,...,...
143855,71.0,31,4
143856,72.0,31,4
143857,72.0,31,3


In [9]:
rad_grp.to_csv("data/ARIA_merged&selected/Rad_grp.csv", index=True)